In [2]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from datasets import Dataset

import matplotlib as plt
import seaborn as sns


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Read froom the files and aggregate the unique data
def process_csv(file_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Filter rows where 'TTL_CLICKS' is greater than 3
    df_1 = df[df['TTL_CLICKS'] > 3].copy()

    # Create a new column 'Corpus' combining other columns
    df_1['Corpus'] = df_1['CAMPGN_NM'] + " " + df_1['LINK_NAME'] + " " + df_1['INTERACTION_DETAIL']



    # Remove duplicate entries based on the 'Corpus' column
    df_1.drop_duplicates(subset='Corpus', inplace=True)

    return df_1

def process_csv1(file_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Filter rows where 'TTL_CLICKS' is greater than 3
    df_1 = df[df['TTL_CLICKS'] > 3].copy()

    # Create a new column 'Corpus' combining other columns
    df_1['Corpus'] = df_1['CAMPGN_SUBJCT_LINE_TXT'] + " " +df_1['CAMPGN_NM'] + " " + df_1['LINK_NAME'] + " " + df_1['INTERACTION_DETAIL']



    # Remove duplicate entries based on the 'Corpus' column
    df_1.drop_duplicates(subset='Corpus', inplace=True)

    return df_1

# List of file paths for CSV files
file_paths = [f"links/AA_links{i}.csv" for i in range(1, 10)]

# Initialize an empty DataFrame to collect all processed data
all_processed_data = pd.DataFrame()

# Process each file and append the result to the master DataFrame
for path in file_paths:
    processed_df = process_csv1(path)
    all_processed_data = pd.concat([all_processed_data, processed_df], ignore_index=True)

# Save the combined processed DataFrame to a new CSV file
all_processed_data.to_csv("all_processed_data1.csv", index=False)
print("All processed data saved to 'all_processed_data.csv'")


/tmp/ipykernel_249346/2072653565.py:21: DtypeWarning: Columns (5,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_249346/2072653565.py:21: DtypeWarning: Columns (5,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_249346/2072653565.py:21: DtypeWarning: Columns (5,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_249346/2072653565.py:21: DtypeWarning: Columns (5,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_249346/2072653565.py:21: DtypeWarning: Columns (5,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
/tmp/ipykernel_249346/2072653565.py:21: DtypeWarning: Columns (5,18) have mixed types. Specify dtype option on import or set low_memory

All processed data saved to 'all_processed_data.csv'


In [3]:
df = pd.read_csv('all_processed_data1.csv')

In [4]:
df.drop_duplicates(subset='Corpus', inplace=True)

In [5]:
# df = df[df['CAMPGN_LANG_NM'].isin(['EN', 'Queens English', 'US English'])]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34056 entries, 0 to 34430
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   MKT_BASE_CAMPGN_ID           34056 non-null  int64 
 1   PARENT_MKT_CAMPGN_ID         34056 non-null  int64 
 2   MKT_CAMPGN_ID                34056 non-null  int64 
 3   CAMPGN_NM                    34055 non-null  object
 4   CAMPGN_SUBJCT_LINE_TXT       34056 non-null  object
 5   CAMPGN_CELL_NM               27313 non-null  object
 6   BUSINES_UNIT_NM              33602 non-null  object
 7   CAMPGN_CATG_CD               33602 non-null  object
 8   PGM_CAMPGN_NM                33442 non-null  object
 9   CAMPGN_LANG_NM               33364 non-null  object
 10  CAMPGN_REGION_CD             33353 non-null  object
 11  CAMPGN_OFFER_TYPE_CD         33515 non-null  object
 12  CAMPGN_OFFER_QTY_CD          30891 non-null  object
 13  CAMPGN_REGSTRTN_REQUIR_IND   33389 n

In [6]:
STOPWORDS = stopwords.words('english')
STOPWORDS =  STOPWORDS  + ['comma', 'na', 'dynamic', 'name', 'aadv']

In [7]:
def preprocess_text(text, stop_words = STOPWORDS):
    # Convert to lowercase
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', ' ', str(text), flags=re.MULTILINE)

    # Remove punctuation
    text = re.sub('[^a-zA-Z]', ' ', str(text))
    text = re.sub(r'_', ' ', str(text))
    text = re.sub(r'71095826-bf22-49bd-9678-7d89ec4f6d49', '', text) 

    # Remove tags
    text = re.sub("&lt;/?.*?&gt;", " ", text)

    # Remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ", text)
    text = re.sub(r'\{[^}]*\}', '', text)

    # Remove stop words
    word_tokens = word_tokenize(text)
    text = ' '.join([w for w in word_tokens if w not in stop_words])

    # Remove words with only two letters
    text = ' '.join([w for w in text.split() if len(w) > 2])

    # Stemming
    stemmer = PorterStemmer()
    word_list = word_tokenize(text)
    text = ' '.join([stemmer.stem(w) for w in word_list])

    # Lemmatization
    lem = WordNetLemmatizer()
    word_list = word_tokenize(text)
    text = ' '.join([lem.lemmatize(w) for w in word_list])

    return text

In [8]:
preprocessed_text = df['Corpus'].apply(preprocess_text)

In [9]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli",
                      device = 0)


2024-08-05 11:17:54.309959: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-05 11:17:54.488150: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
preprocessed_text =preprocessed_text.reset_index()

In [11]:
for i in range(len(preprocessed_text)):
    if not preprocessed_text['Corpus'][i]:  
        preprocessed_text['Corpus'][i] = ' '

In [95]:
# preprocessed_subjline = preprocessed_subjline['CAMPGN_SUBJCT_LINE_TXT']

In [12]:
preprocessed_text = preprocessed_text['Corpus']
preprocessed_text

0        updat eqd waiver progress cobrand eqd trigger ...
1        updat eqd waiver progress cobrand eqd trigger ...
2        updat eqd waiver progress cobrand eqd trigger ...
3        updat eqd waiver progress cobrand eqd trigger ...
4        updat eqd waiver progress cobrand eqd trigger ...
                               ...                        
34051           ota recur send ota thank final nav get app
34052         ota recur send ota thank final updat address
34053       ota recur send ota thank final android appstor
34054    ota recur send ota thank final link custom ser...
34055    ota recur send ota thank final nav privaci polici
Name: Corpus, Length: 34056, dtype: object

In [13]:
print(type(preprocessed_text))


<class 'pandas.core.series.Series'>


In [14]:
preprocessed_text_ds = Dataset.from_pandas(preprocessed_text.to_frame(name='text'))

In [17]:
candidate_labels = ['psoitive','negative']
hypothesis_template = "The sentiment is {}."

classified_scores = []

def classify_texts(batch):
    results = classifier(batch['text'], candidate_labels=['positive', 'negative'], hypothesis_template="The sentiment is {}.")
    return {'labels': [result['labels'] for result in results], 'scores': [result['scores'] for result in results]}

classified_scores = preprocessed_text_ds.map(classify_texts, batched=True, batch_size=32)


Map:   0%|          | 0/34056 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [23]:
df[['Corpus_POSITIVE','Corpus_NEGATIVE']]=classified_scores['scores']
df['Preprocessed_text'] = preprocessed_text

In [25]:
df.to_csv('SubjLineLink_Corpus_sentiment.csv')

In [26]:
df.head(10)

,MKT_BASE_CAMPGN_ID,PARENT_MKT_CAMPGN_ID,MKT_CAMPGN_ID,CAMPGN_NM,CAMPGN_SUBJCT_LINE_TXT,CAMPGN_CELL_NM,BUSINES_UNIT_NM,CAMPGN_CATG_CD,PGM_CAMPGN_NM,CAMPGN_LANG_NM,...,FIRST_INTERACTION_DATE_TIME,LAST_INTERACTION_DATE_TIME,NUM_LYLTY_CUST,NUM_ALL_CUST,TTL_CLICKS,BUCKET,Corpus,Corpus_POSITIVE,Corpus_NEGATIVE,Preprocessed_text
0,106977,109747,109748,Cobrand 15K EQD Trigger - Halfway,Update on your EQD waiver progress,Cobrand 15K EQD Trigger - Halfway_ENG,AAdvantage,Co-Brand,AAdv Barclays - ECM,US English,...,01/01/2022 13:19:24:323000,12/31/2021 13:18:14:640000,41,41,49,1,Update on your EQD waiver progress Cobrand 15K...,0.572058,0.427942,updat eqd waiver progress cobrand eqd trigger ...
1,106977,109747,109748,Cobrand 15K EQD Trigger - Halfway,Update on your EQD waiver progress,Cobrand 15K EQD Trigger - Halfway_ENG,AAdvantage,Co-Brand,AAdv Barclays - ECM,US English,...,12/31/2021 13:36:50:977000,12/31/2021 14:47:20:700000,5,5,5,1,Update on your EQD waiver progress Cobrand 15K...,0.757528,0.242472,updat eqd waiver progress cobrand eqd trigger ...
2,106977,109747,109748,Cobrand 15K EQD Trigger - Halfway,Update on your EQD waiver progress,Cobrand 15K EQD Trigger - Halfway_ENG,AAdvantage,Co-Brand,AAdv Barclays - ECM,US English,...,01/01/2022 19:51:03:177000,12/31/2021 13:22:16:280000,3,3,4,1,Update on your EQD waiver progress Cobrand 15K...,0.623497,0.376503,updat eqd waiver progress cobrand eqd trigger ...
3,106977,109747,109748,Cobrand 15K EQD Trigger - Halfway,Update on your EQD waiver progress,Cobrand 15K EQD Trigger - Halfway_ENG,AAdvantage,Co-Brand,AAdv Barclays - ECM,US English,...,01/01/2022 10:17:12:670000,12/31/2021 09:43:31:543000,15,15,16,1,Update on your EQD waiver progress Cobrand 15K...,0.956717,0.043283,updat eqd waiver progress cobrand eqd trigger ...
4,106977,109747,109748,Cobrand 15K EQD Trigger - Halfway,Update on your EQD waiver progress,Cobrand 15K EQD Trigger - Halfway_ENG,AAdvantage,Co-Brand,AAdv Barclays - ECM,US English,...,01/01/2022 09:17:43:173000,12/31/2021 20:06:29:547000,4,4,8,1,Update on your EQD waiver progress Cobrand 15K...,0.716555,0.283445,updat eqd waiver progress cobrand eqd trigger ...
5,109161,109751,109752,Cobrand 30K EQD Trigger - Completion,Congratulations! You've qualified for an EQD w...,Cobrand 30K EQD Trigger - Completion_ENG,AAdvantage,Co-Brand,AAdv Barclays - ECM,US English,...,01/01/2022 01:13:08:913000,12/31/2021 20:46:46:640000,15,15,24,1,Congratulations! You've qualified for an EQD w...,0.998258,0.001742,congratul qualifi eqd waiver cobrand eqd trigg...
6,109161,109751,109752,Cobrand 30K EQD Trigger - Completion,Congratulations! You've qualified for an EQD w...,Cobrand 30K EQD Trigger - Completion_ENG,AAdvantage,Co-Brand,AAdv Barclays - ECM,US English,...,01/01/2022 01:08:15:670000,12/31/2021 23:38:19:417000,157,157,201,1,Congratulations! You've qualified for an EQD w...,0.998460,0.001540,congratul qualifi eqd waiver cobrand eqd trigg...
7,109161,109751,109752,Cobrand 30K EQD Trigger - Completion,Congratulations! You've qualified for an EQD w...,Cobrand 30K EQD Trigger - Completion_ENG,AAdvantage,Co-Brand,AAdv Barclays - ECM,US English,...,12/30/2021 13:11:39:360000,12/30/2021 23:10:23:627000,4,4,5,1,Congratulations! You've qualified for an EQD w...,0.998438,0.001562,congratul qualifi eqd waiver cobrand eqd trigg...
8,109161,109751,109752,Cobrand 30K EQD Trigger - Completion,Congratulations! You've qualified for an EQD w...,Cobrand 30K EQD Trigger - Completion_ENG,AAdvantage,Co-Brand,AAdv Barclays - ECM,US English,...,12/29/2021 13:17:12:710000,12/30/2021 16:52:09:057000,4,4,7,1,Congratulations! You've qualified for an EQD w...,0.998485,0.001515,congratul qualifi eqd waiver cobrand eqd trigg...
9,114169,114307,114309,Hotel_Planner_Prep - 5 day - Updated 20220321,Exclusivas ofertas de hoteles para su próximo ...,Hotel_Planner_Prep - 5 day - Updated 20220321_SP,Loyalty Non Card Partner,Partner - Hotel,AAdv Hotel - Bookingcom,NaN,...,07/19/2022 20:34:51:143000,07/25/2022 19:12:27:363